<a href="https://colab.research.google.com/github/rero2h/T5/blob/main/Copy_of_YOLO_Practical_Exam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Road Traffic Vehicle Counting Exam

In this exam, you will use the YOLO (You Only Look Once) object detection model to analyze a video of road traffic. Your primary task is to count the number of vehicles exiting the road at each of the four exits (located at the top, bottom, left, and right of the frame). The video for this task is provided at `Datasets/YOLO_Exam_Video.mp4`.

## Objectives
- Load and process the video using OpenCV.
- Utilize the YOLO model to detect vehicles in each frame.
- Manipulate the video frames using OpenCV.
- Track vehicles as they move across the video frames.
- Count the number of vehicles exiting via the top street.
- Count the number of vehicles exiting via the bottom street.
- Count the number of vehicles exiting via the left street.
- Count the number of vehicles exiting via the right street.


## Setup Environment

Before you begin, ensure you have the necessary libraries installed. You will need `opencv`, and `ultralytics` among others.
If these are not installed, you should install them.

In [1]:
!pip install ultralytics -q

In [2]:
import cv2
from ultralytics import YOLO, solutions

## Load the YOLO Model

You will first need to load the YOLO model, there is a custom model for this exam called `YOLO_Model.pt` under Datasets file ⚠️ PLEASE USE THIS MODEL AND DO NOT USE ANY OTHER MODEL ⚠️.
Write the code to load the YOLO model below:

In [3]:
model = YOLO('YOLO_Model.pt')

## Prepare the Video Capture

Create a variable to capture the video frames, you can use `cv2.VideoCapture()` to achive this.

In [4]:
cap = cv2.VideoCapture('/content/YOLO_Video.mp4')
assert cap.isOpened(), 'Error reading video file'

## Get Video Information

You can use `cv2` library to get these information fro the `VideoCapture()` variable you created to extract these information:
* `height`: Video's height.
* `width`: Video's width.
* `fps`: Video's frames.

In [5]:
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

## Prepare Video Writer to Store the Output

Create a variable that uses `cv2.VideoCapture()` to save the video with the bounding boxes, specified Region of Interest rectangle (ROI) and the counted vehicles on each sides. You will need to make the video with the same `fps`, `width`, `height`, and specify the codec and output path of the video.

In [6]:
video_writer = cv2.VideoWriter('YOLO_Practical_Exam_output.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))

## Expermint to get the `(x1, y1, x2, y2)` of the ROI

### Hint
When defining the Regions of Interest (ROIs) for detecting vehicles exiting the road, consider the coordinates that outline the relevant areas. These coordinates can be specified as `(x1, y1, x2, y2)`, where `(x1, y1)` is the top-left corner and `(x2, y2)` is the bottom-right corner of the rectangle.

Experiment with different coordinates to accurately cover the streets where vehicles exit. Visualize these ROIs by drawing rectangles on a frame of the video and adjust the coordinates as needed until you capture the desired areas effectively.

In [7]:
success, frame = cap.read()
x1, y1, x2, y2 = 100, 200, 300, 400
cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

array([[[ 41,  77,  72],
        [ 54,  90,  85],
        [ 38,  79,  73],
        ...,
        [ 55,  51,  47],
        [ 55,  51,  47],
        [ 55,  51,  47]],

       [[ 22,  58,  53],
        [ 49,  85,  80],
        [ 33,  74,  68],
        ...,
        [ 56,  52,  48],
        [ 55,  51,  47],
        [ 55,  51,  47]],

       [[  6,  42,  37],
        [ 40,  76,  71],
        [ 34,  75,  69],
        ...,
        [ 55,  51,  47],
        [ 55,  51,  47],
        [ 54,  50,  46]],

       ...,

       [[128, 118, 194],
        [111, 101, 177],
        [ 99,  87, 163],
        ...,
        [108, 117, 123],
        [108, 117, 123],
        [108, 117, 123]],

       [[117, 100, 183],
        [101,  84, 167],
        [112, 100, 164],
        ...,
        [110, 119, 125],
        [108, 117, 123],
        [108, 117, 123]],

       [[ 99,  82, 165],
        [106,  89, 172],
        [126, 114, 178],
        ...,
        [110, 119, 125],
        [110, 119, 125],
        [110, 119, 125]]

## Process Video Frames and Identify Vehicles Left the Road Form Each Street

For each frame in the video, use the YOLO model to detect and track vehicles. You will need to write a loop that processes each frame and applies the YOLO model.

In each frame, after detecting the vehicles, determine whether they exited the road via the top, bottom, left, or right street. You can use the positions of the bounding boxes provided by YOLO to do this.

- The video should display bounding boxes around the detected objects.
- The video should display your name on the top.
- It should display the calculated center of each vehicle.
- The video should display the confidence score, alongside the object ID and class ID of each detected and tracked object.
- The video should display the rectangles representing the regions where you count the vehicles that have crossed and exited.
- The video should display the number of vehicles that exited via the top street.
- The video should display the number of vehicles that exited via the bottom street.
- The video should display the number of vehicles that exited via the left street.
- The video should display the number of vehicles that exited via the right street.

In [8]:
mid_y = h // 2
line_points = [(20, mid_y), (w - 20, mid_y)]

counter = solutions.ObjectCounter(
    view_img=True,
    reg_pts=line_points,
    names=model.names,
    draw_tracks=True,
    line_thickness=2,
)


while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        print('Video frame is empty or video processing has been successfully completed.')
        break

    tracks = model.track(im0, persist=True, show=False)


    im0 = counter.start_counting(im0, tracks)
    video_writer.write(im0)

WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()

Line Counter Initiated.

0: 384x640 2 buss, 82 cars, 2 trucks, 2 vans, 7396.7ms
Speed: 4.9ms preprocess, 7396.7ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 82 cars, 2 trucks, 2 vans, 2978.1ms
Speed: 4.1ms preprocess, 2978.1ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 82 cars, 2 trucks, 2 vans, 2695.8ms
Speed: 3.8ms preprocess, 2695.8ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 82 cars, 2 trucks, 2 vans, 2818.8ms
Speed: 6.3ms preprocess, 2818.8ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 82 cars, 2 trucks, 2 vans, 3861.7ms
Speed: 3.8ms preprocess, 3861.7ms inference, 3.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 82 cars, 2 trucks, 2 vans, 5558.1ms
Speed: 6.2ms preprocess, 5558.1ms inference, 2.7ms postprocess per imag

In [9]:
cap.release()
video_writer.release()
cv2.destroyAllWindows()

## Save and Submit Your Work

Submit both the notebook and the output video